In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import fcluster

In [2]:
# Read vcf and coverage file for each population
filepath = 'G:\\Dropbox (Vetsigian lab)\\Vetsigian lab Team Folder\\Ye\\Genomics\\Community mutants\\'
folders = ['PopA', 'PopB', 'PopC', 'PopD', 'PopE', 'PopF', 'PopG', 'PopH']
vcfA = pd.read_csv(filepath + folders[0] + '\\vcfs_A.csv')
covA = pd.read_csv(filepath + folders[0] + '\\coverage_popA.csv')

vcfB = pd.read_csv(filepath + folders[1] + '\\vcfs_B.csv')
covB = pd.read_csv(filepath + folders[1] + '\\coverage_popB.csv')

vcfC = pd.read_csv(filepath + folders[2] + '\\vcfs_C.csv')
covC = pd.read_csv(filepath + folders[2] + '\\coverage_popC.csv')

vcfD = pd.read_csv(filepath + folders[3] + '\\vcfs_D.csv')
covD = pd.read_csv(filepath + folders[3] + '\\coverage_popD.csv')

vcfE = pd.read_csv(filepath + folders[4] + '\\vcfs_E.csv')
covE = pd.read_csv(filepath + folders[4] + '\\coverage_popE.csv')

vcfF = pd.read_csv(filepath + folders[5] + '\\vcfs_F.csv')
covF = pd.read_csv(filepath + folders[5] + '\\coverage_popF.csv')

vcfG = pd.read_csv(filepath + folders[6] + '\\vcfs_G.csv')
covG = pd.read_csv(filepath + folders[6] + '\\coverage_popG.csv')

vcfH = pd.read_csv(filepath + folders[7] + '\\vcfs_H.csv')
covH = pd.read_csv(filepath + folders[7] + '\\coverage_popH.csv')

In [3]:
vcfH.head(4)

,Unnamed: 0,Name,Minimum,Maximum,Sequence,locus_tag,Protein Effect,note,1H VF,1H Coverage,...,3H VF,3H Coverage,4H VF,4H Coverage,6H VF,6H Coverage,7H VF,7H Coverage,8H VF,8H Coverage
0,0,G,104233.0,104233.0,A,SCO0124,None,"SCJ21.05, unknown, len: 453 aa; similar in par...",0.962,53.0,...,0.979,48.0,1.0,60.0,1.000,57.0,1.00,78.0,NaN,NaN
1,1,G,603360.0,603360.0,A,Intergenetic region,NaN,NaN,0.963,54.0,...,1.000,52.0,1.0,50.0,0.986,71.0,1.00,89.0,NaN,NaN
2,2,G,619172.0,619172.0,C,SCO0577,None,"SCF55.01c, hypothetical protein, len: >231 aa;...",1.000,37.0,...,0.984,62.0,1.0,54.0,0.987,79.0,0.99,104.0,NaN,NaN
3,3,G,619978.0,619978.0,C,SCO0578,None,"SCF55.02c, possible triosephosphate isomerase,...",1.000,29.0,...,NaN,NaN,1.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
covH.head(4)

,Unnamed: 0,Position,H0,H1,H2,H3,H4,H6,H7,H8
0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
def fill_coverage_info_in_vcfs(vcf, coverage_file):
    '''
    Inputs:
    vcf = variance file of the query population (i.e. 'vcfs_A') where variance frequency and corresponding coverage 
          at certain generation need to be filled using information from coverage file
    coverage_file = the coverage at each postion for teh query population across the generations
    
    Output:
    vcf = variance file of the query population where the coverage info of the population was filled  
    '''
    colnames_vcf = vcf.columns
    colvf = colnames_vcf[8::2]
    colcov = colnames_vcf[9::2]

    colnames_cov = coverage_file.columns
    colcov_ = colnames_cov[3:]

    for col, col_ in zip(colcov, colcov_):
        mask = vcf[col].isna()
        min_pos = vcf.loc[mask]['Minimum']
        max_pos = vcf.loc[mask]['Maximum']
        range_pos = [np.arange(l,r+1,1) for l,r in zip(min_pos, max_pos)]
        coverage_mean = []
        for pos in range_pos:
            pos_mask = coverage_file['Position'].isin(pos)
            coverage = coverage_file.loc[pos_mask][col_]
            coverage_mean.append(round(np.mean(coverage), 0))
        vcf.loc[mask, col] = coverage_mean

    vcf[colvf] = vcf[colvf].fillna(0)
    for colc, colv in zip(colcov, colvf):
        cov0_mask = np.sum(vcf[[colv, colc]], axis =1) == 0
        vcf.loc[cov0_mask, colv] = np.nan
    vcf = vcf.drop(colnames_vcf[0], axis = 1)
    return vcf

In [10]:
savepath = 'G:\\Dropbox (Vetsigian lab)\\Vetsigian lab Team Folder\\Ye\\Genomics\\Community mutants\\'
folders = ['PopA', 'PopB', 'PopC', 'PopD', 'PopE', 'PopF', 'PopG', 'PopH']

In [19]:
len(vcfB)

221976

In [11]:
vcfA_filled = fill_coverage_info_in_vcfs(vcfA, covA)
vcfA_filled.to_csv(savepath + folders[0] + '\\vcfA_filled.csv')

In [ ]:
vcfB_filled = fill_coverage_info_in_vcfs(vcfB, covB)
vcfB_filled.to_csv(savepath + folders[1] + '\\vcfB_filled.csv')

In [13]:
vcfC_filled = fill_coverage_info_in_vcfs(vcfC, covC)
vcfC_filled.to_csv(savepath + folders[2] + '\\vcfC_filled.csv')

In [14]:
vcfD_filled = fill_coverage_info_in_vcfs(vcfD, covD)
vcfD_filled.to_csv(savepath + folders[3] + '\\vcfD_filled.csv')

In [ ]:
vcfE_filled = fill_coverage_info_in_vcfs(vcfE, covE)
vcfE_filled.to_csv(savepath + folders[4] + '\\vcfE_filled.csv')

In [ ]:
vcfF_filled = fill_coverage_info_in_vcfs(vcfF, covF)
vcfF_filled.to_csv(savepath + folders[5] + '\\vcfF_filled.csv')

In [ ]:
vcfG_filled = fill_coverage_info_in_vcfs(vcfG, covG)
vcfG_filled.to_csv(savepath + folders[6] + '\\vcfG_filled.csv')

In [6]:
vcfH_filled = fill_coverage_info_in_vcfs(vcfH, covH)
vcfH_filled.to_csv(savepath + folders[7] + '\\vcfH_filled.csv')

KeyboardInterrupt: 

In [8]:
vcfB_filled.head(5)

NameError: name 'vcfB_filled' is not defined